# Notes

This architecture was described in "Deep learning with convolutional neural networks for brain mapping and decoding of movement-related information from the human EEG", by R. T. Schmirrmester et al, 2018. In this notebook we conduct experimetns showing the importance of using batchnorm/dropout layers, and dependacy between accuracy and the number of timestamps in a sample.

# Set up the environment

In [20]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from __future__ import absolute_import, division, print_function, unicode_literals

# import tf
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K

# import os functions
import os
import time

import numpy as np
import matplotlib.pyplot as plt

from IPython import display

In [4]:
cd ..

/Users/sriramsonti/Desktop/project_C247


# Load the data

### Read the files

In [22]:
X_test = np.load("./EEG_data/X_test.npy")
y_test = np.load("./EEG_data/y_test.npy") - 769
person_train_valid = np.load("./EEG_data/person_train_valid.npy")
X_train_valid = np.load("./EEG_data/X_train_valid.npy")
y_train_valid = np.load("./EEG_data/y_train_valid.npy") - 769
person_test = np.load("./EEG_data/person_test.npy")

### Shape of data

In [23]:
print("training/Valid data shape: {}".format(X_train_valid.shape))       # training data of many persons
print("Test data shape: {}".format(X_test.shape))                        # test data of many persons
print("Training/Valid target shape: {}".format(y_train_valid.shape))     # training labels of many persons
print("Test target shape: {}".format(y_test.shape))                      # test labels of many persons
print("Person train/valid  shape: {}".format(person_train_valid.shape))  # which person correspond to the trail in test set
print("Person test shape: {}".format(person_test.shape))                 # which person correspond to the trail in test set

training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid  shape: (2115, 1)
Person test shape: (443, 1)


### divide dataset into training and validation

In [24]:
perm = np.random.permutation(X_train_valid.shape[0])
num_train = int(0.8 * X_train_valid.shape[0])
num_valid = X_train_valid.shape[0] - num_train
X_train =  X_train_valid[perm[0:num_train]]
y_train =  y_train_valid[perm[0:num_train]]
X_valid = X_train_valid[perm[num_train: ]]
y_valid = y_train_valid[perm[num_train: ]]


print("Training data shape: {}".format(X_train.shape))
print("Training label shape: {}".format(y_train.shape))
print("Validation data shape: {}".format(X_valid.shape))
print("Validation label shape: {}".format(y_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Test label shape: {}".format(y_test.shape))

Training data shape: (1692, 22, 1000)
Training label shape: (1692,)
Validation data shape: (423, 22, 1000)
Validation label shape: (423,)
Test data shape: (443, 22, 1000)
Test label shape: (443,)


### Augmented dataset

In [25]:
def sliding_window(X_arr, y_arr, time_window=100, time_step=1, time_stride=1):
    temp_x = np.moveaxis(X_arr, 2, 0)
    temp_x = temp_x.astype(np.float32)
    buff = []
    
    num_slices = (len(temp_x)-time_window*time_step) // time_stride + 1
    
    # get time slices for data
    for i in range(num_slices):
        buff.append(temp_x[i*time_stride:i*time_stride + time_window*time_step:time_step])
        buff[i] = np.moveaxis(buff[i], 0, 2)
        # uncomment this if additional dimension is needed
        # buff[i] = buff[i].reshape(1, buff[i].shape[0], buff[i].shape[1], buff[i].shape[2])
        
    temp_x = np.concatenate(buff)
        
    # get time slice for labels
    temp_y = np.ones((X_arr.shape[0],num_slices))
    
    for i in range(len(y_arr)):
        temp_y[i] = temp_y[i] * y_arr[i]
        
    temp_y = temp_y.reshape((-1))
    
    return temp_x, temp_y

# Experiment 1: naive implementation of deep model

In this experiment, we show that naive implementation of deep convolutional model can not return good results because of overfit.

### Construct model

In [26]:
# input
deep_input = layers.Input(shape=(22, 1000))



# ================================== CONV1 ================================== #

# conv accross time domain
r1 = layers.Reshape((22, 1000, 1))(deep_input)
c1 = layers.Conv2D(25, (1, 10), strides=(1, 1), activation="elu")(r1)
t1 = tf.keras.layers.Permute((2, 3, 1))(c1)

r2 = layers.Reshape((991, 25*22, 1))(t1)
c2 = layers.Conv2D(25, (1, 25*22), strides=(1, 1), activation="elu")(r2)

# max pool across time domain
r3 = layers.Reshape((991, 25, 1))(c2)
maxpool3 = layers.MaxPooling2D(pool_size=(3, 1), strides=(3, 1))(r3)

# =========================================================================== #



# ================================= CONV2-4 ================================= #

c4 = layers.Conv2D(50, (10, 25), strides=(1, 1), activation="elu")(maxpool3)
r4 = layers.Reshape((321, 50, 1))(c4)
maxpool4 = layers.MaxPooling2D(pool_size=(3, 1), strides=(3, 1))(r4)

c5 = layers.Conv2D(100, (10, 50), strides=(1, 1), activation="elu")(maxpool4)
r5 = layers.Reshape((98, 100, 1))(c5)
maxpool5 = layers.MaxPooling2D(pool_size=(3, 1), strides=(3, 1))(r5)

c6 = layers.Conv2D(200, (10, 100), strides=(1, 1), activation="elu")(maxpool5)
r6 = layers.Reshape((23, 200, 1))(c6)
maxpool6 = layers.MaxPooling2D(pool_size=(3, 1), strides=(3, 1))(r6)

# =========================================================================== #

f7 = layers.Flatten()(r6)

# output
deep_output = layers.Dense(4, activation="softmax")(f7)

In [27]:
deep_model = keras.Model(inputs = deep_input, outputs = deep_output, name="deep_model")
deep_model.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

In [28]:
deep_model.summary()

Model: "deep_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 22, 1000)]        0         
_________________________________________________________________
reshape_6 (Reshape)          (None, 22, 1000, 1)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 991, 25)       275       
_________________________________________________________________
permute_1 (Permute)          (None, 991, 25, 22)       0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 991, 550, 1)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 991, 1, 25)        13775     
_________________________________________________________________
reshape_8 (Reshape)          (None, 991, 25, 1)        0

### Make checkpoints

In [29]:
# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/deep_model_1000',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

### Train model on single person

In [30]:
person_num = 0
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

Training data shape for 1 person: (189, 22, 1000)
Training label shape for 1 person: (189,)
Validation data shape for 1 person: (48, 22, 1000)
Validation label shape for 1 person: (48,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)


In [31]:
for my_lr in [0.005, 0.001, 0.0005, 0.0001]:
    for num_epochs in [10, 20, 30]:
        print(f"Training with lr: {my_lr} and num_epochs = {num_epochs}")
        deep_model_single = keras.Model(inputs = deep_input, outputs = deep_output, name="deep_model")
        deep_model_single.compile("adam", "sparse_categorical_crossentropy", lr = my_lr, metrics=["acc"])
        deep_model_single.fit(single_person_X_train, single_person_y_train,
                                                    validation_data = (single_person_X_valid, single_person_y_valid),
                                                    epochs = num_epochs)
        deep_model_single.evaluate(single_person_X_test, single_person_y_test)
        deep_model_single.evaluate(X_test, y_test)

Training with lr: 0.005 and num_epochs = 10
Train on 189 samples, validate on 48 samples
Epoch 1/10
189/189 [==============================] - 14s 76ms/sample - loss: 1.3888 - acc: 0.2116 - val_loss: 1.3950 - val_acc: 0.1875
Epoch 2/10
189/189 [==============================] - 10s 54ms/sample - loss: 1.3835 - acc: 0.2698 - val_loss: 1.4275 - val_acc: 0.1875
Epoch 3/10
189/189 [==============================] - 10s 55ms/sample - loss: 1.3755 - acc: 0.2804 - val_loss: 1.4194 - val_acc: 0.1667
Epoch 4/10
189/189 [==============================] - 10s 54ms/sample - loss: 1.2554 - acc: 0.4815 - val_loss: 2.5132 - val_acc: 0.3333
Epoch 5/10
189/189 [==============================] - 10s 53ms/sample - loss: 1.1905 - acc: 0.4974 - val_loss: 1.5100 - val_acc: 0.3125
Epoch 6/10
189/189 [==============================] - 12s 64ms/sample - loss: 1.1709 - acc: 0.5503 - val_loss: 1.6358 - val_acc: 0.2708
Epoch 7/10
189/189 [==============================] - 11s 57ms/sample - loss: 1.0046 - acc: 0.5

Training with lr: 0.005 and num_epochs = 20
Train on 189 samples, validate on 48 samples
Epoch 1/20
189/189 [==============================] - 12s 63ms/sample - loss: 0.7737 - acc: 0.6720 - val_loss: 2.0067 - val_acc: 0.2917
Epoch 2/20
189/189 [==============================] - 10s 54ms/sample - loss: 0.7196 - acc: 0.7831 - val_loss: 1.9283 - val_acc: 0.3333
Epoch 3/20
189/189 [==============================] - 10s 55ms/sample - loss: 0.4577 - acc: 0.8571 - val_loss: 3.4542 - val_acc: 0.3125
Epoch 4/20
189/189 [==============================] - 11s 59ms/sample - loss: 0.2325 - acc: 0.9524 - val_loss: 3.4534 - val_acc: 0.2708
Epoch 5/20
189/189 [==============================] - 10s 54ms/sample - loss: 0.1860 - acc: 0.9683 - val_loss: 3.7056 - val_acc: 0.2708
Epoch 6/20
189/189 [==============================] - 12s 62ms/sample - loss: 0.0571 - acc: 1.0000 - val_loss: 4.8209 - val_acc: 0.3542
Epoch 7/20
189/189 [==============================] - 10s 54ms/sample - loss: 0.0203 - acc: 1.0

Training with lr: 0.005 and num_epochs = 30
Train on 189 samples, validate on 48 samples
Epoch 1/30
189/189 [==============================] - 12s 62ms/sample - loss: 0.0038 - acc: 1.0000 - val_loss: 9.7646 - val_acc: 0.3333
Epoch 2/30
189/189 [==============================] - 10s 54ms/sample - loss: 0.0469 - acc: 0.9894 - val_loss: 8.8996 - val_acc: 0.2917
Epoch 3/30
189/189 [==============================] - 10s 54ms/sample - loss: 0.0390 - acc: 0.9894 - val_loss: 6.8451 - val_acc: 0.2708
Epoch 4/30
189/189 [==============================] - 11s 56ms/sample - loss: 0.0933 - acc: 0.9683 - val_loss: 4.9362 - val_acc: 0.3125
Epoch 5/30
189/189 [==============================] - 10s 53ms/sample - loss: 0.0782 - acc: 0.9841 - val_loss: 4.0754 - val_acc: 0.3125
Epoch 6/30
189/189 [==============================] - 10s 53ms/sample - loss: 0.0815 - acc: 0.9841 - val_loss: 3.8603 - val_acc: 0.3125
Epoch 7/30
189/189 [==============================] - 10s 53ms/sample - loss: 0.0608 - acc: 0.9

Training with lr: 0.001 and num_epochs = 10
Train on 189 samples, validate on 48 samples
Epoch 1/10
189/189 [==============================] - 12s 65ms/sample - loss: 1.5194e-06 - acc: 1.0000 - val_loss: 37.9157 - val_acc: 0.3125
Epoch 2/10
189/189 [==============================] - 10s 53ms/sample - loss: 1.0124e-04 - acc: 1.0000 - val_loss: 24.1110 - val_acc: 0.3333
Epoch 3/10
189/189 [==============================] - 11s 57ms/sample - loss: 0.0013 - acc: 1.0000 - val_loss: 18.1166 - val_acc: 0.3333
Epoch 4/10
189/189 [==============================] - 10s 54ms/sample - loss: 0.3229 - acc: 0.9524 - val_loss: 18.2648 - val_acc: 0.3750
Epoch 5/10
189/189 [==============================] - 10s 53ms/sample - loss: 0.6760 - acc: 0.9206 - val_loss: 13.7992 - val_acc: 0.3125
Epoch 6/10
189/189 [==============================] - 10s 54ms/sample - loss: 0.0649 - acc: 0.9894 - val_loss: 10.0120 - val_acc: 0.2500
Epoch 7/10
189/189 [==============================] - 10s 53ms/sample - loss: 0.1

Training with lr: 0.001 and num_epochs = 20
Train on 189 samples, validate on 48 samples
Epoch 1/20
189/189 [==============================] - 12s 63ms/sample - loss: 0.0420 - acc: 0.9841 - val_loss: 13.0388 - val_acc: 0.2917
Epoch 2/20
189/189 [==============================] - 10s 54ms/sample - loss: 0.5008 - acc: 0.9259 - val_loss: 8.8385 - val_acc: 0.2917
Epoch 3/20
189/189 [==============================] - 10s 54ms/sample - loss: 0.0226 - acc: 0.9947 - val_loss: 5.6294 - val_acc: 0.2500
Epoch 4/20
189/189 [==============================] - 11s 58ms/sample - loss: 0.0192 - acc: 0.9947 - val_loss: 5.8104 - val_acc: 0.2708
Epoch 5/20
189/189 [==============================] - 10s 54ms/sample - loss: 0.0040 - acc: 1.0000 - val_loss: 7.6094 - val_acc: 0.2917
Epoch 6/20
189/189 [==============================] - 10s 54ms/sample - loss: 4.3070e-04 - acc: 1.0000 - val_loss: 9.1177 - val_acc: 0.2917
Epoch 7/20
189/189 [==============================] - 10s 54ms/sample - loss: 9.4272e-05 -

...

Training with lr: 0.0005 and num_epochs = 20
Train on 189 samples, validate on 48 samples
Epoch 1/20
189/189 [==============================] - 12s 62ms/sample - loss: 0.1702 - acc: 0.9735 - val_loss: 22.0298 - val_acc: 0.3333
Epoch 2/20
189/189 [==============================] - 10s 54ms/sample - loss: 0.3296 - acc: 0.9524 - val_loss: 14.3538 - val_acc: 0.3750
Epoch 3/20
189/189 [==============================] - 10s 54ms/sample - loss: 0.1238 - acc: 0.9683 - val_loss: 20.6541 - val_acc: 0.2708
Epoch 4/20
189/189 [==============================] - 11s 57ms/sample - loss: 0.0090 - acc: 0.9947 - val_loss: 23.9335 - val_acc: 0.2917
Epoch 5/20
189/189 [==============================] - 10s 53ms/sample - loss: 2.5289e-05 - acc: 1.0000 - val_loss: 27.4733 - val_acc: 0.2917
Epoch 6/20
189/189 [==============================] - 11s 56ms/sample - loss: 3.2478e-05 - acc: 1.0000 - val_loss: 30.0376 - val_acc: 0.3125
Epoch 7/20
189/189 [==============================] - 10s 53ms/sample - loss: 4.

Training with lr: 0.0005 and num_epochs = 30
Train on 189 samples, validate on 48 samples
Epoch 1/30
189/189 [==============================] - 12s 61ms/sample - loss: 9.4611e-09 - acc: 1.0000 - val_loss: 35.6063 - val_acc: 0.3125
Epoch 2/30
189/189 [==============================] - 10s 54ms/sample - loss: 1.4507e-08 - acc: 1.0000 - val_loss: 36.5555 - val_acc: 0.2500
Epoch 3/30
189/189 [==============================] - 11s 58ms/sample - loss: 2.1014e-05 - acc: 1.0000 - val_loss: 45.9202 - val_acc: 0.2917
Epoch 4/30
189/189 [==============================] - 10s 54ms/sample - loss: 0.0194 - acc: 0.9947 - val_loss: 63.5664 - val_acc: 0.3542
Epoch 5/30
189/189 [==============================] - 10s 54ms/sample - loss: 0.0108 - acc: 0.9947 - val_loss: 57.9600 - val_acc: 0.3542
Epoch 6/30
189/189 [==============================] - 10s 55ms/sample - loss: 0.1278 - acc: 0.9841 - val_loss: 54.5525 - val_acc: 0.2917
Epoch 7/30
189/189 [==============================] - 11s 57ms/sample - loss

Training with lr: 0.0001 and num_epochs = 10
Train on 189 samples, validate on 48 samples
Epoch 1/10
189/189 [==============================] - 12s 61ms/sample - loss: 2.7122e-08 - acc: 1.0000 - val_loss: 37.8561 - val_acc: 0.2917
Epoch 2/10
189/189 [==============================] - 10s 55ms/sample - loss: 6.3074e-10 - acc: 1.0000 - val_loss: 44.1843 - val_acc: 0.3125
Epoch 3/10
189/189 [==============================] - 11s 57ms/sample - loss: 2.5229e-08 - acc: 1.0000 - val_loss: 56.3581 - val_acc: 0.3125
Epoch 4/10
189/189 [==============================] - 10s 53ms/sample - loss: 0.0017 - acc: 1.0000 - val_loss: 65.0766 - val_acc: 0.2292
Epoch 5/10
189/189 [==============================] - 10s 54ms/sample - loss: 0.0051 - acc: 0.9947 - val_loss: 49.3373 - val_acc: 0.2292
Epoch 6/10
189/189 [==============================] - 11s 59ms/sample - loss: 0.0153 - acc: 0.9894 - val_loss: 49.8184 - val_acc: 0.3958
Epoch 7/10
189/189 [==============================] - 11s 58ms/sample - loss

Training with lr: 0.0001 and num_epochs = 20
Train on 189 samples, validate on 48 samples
Epoch 1/20
189/189 [==============================] - 12s 65ms/sample - loss: 0.0231 - acc: 0.9947 - val_loss: 39.2937 - val_acc: 0.2500
Epoch 2/20
189/189 [==============================] - 10s 55ms/sample - loss: 0.2879 - acc: 0.9841 - val_loss: 42.3717 - val_acc: 0.2708
Epoch 3/20
189/189 [==============================] - 10s 54ms/sample - loss: 2.5452e-04 - acc: 1.0000 - val_loss: 56.1957 - val_acc: 0.2292
Epoch 4/20
189/189 [==============================] - 10s 54ms/sample - loss: 6.2489e-04 - acc: 1.0000 - val_loss: 63.4678 - val_acc: 0.2083
Epoch 5/20
189/189 [==============================] - 10s 54ms/sample - loss: 5.3130e-05 - acc: 1.0000 - val_loss: 67.2302 - val_acc: 0.2083
Epoch 6/20
189/189 [==============================] - 10s 54ms/sample - loss: 5.5239e-05 - acc: 1.0000 - val_loss: 69.8346 - val_acc: 0.1875
Epoch 7/20
189/189 [==============================] - 11s 58ms/sample - 

Training with lr: 0.0001 and num_epochs = 30
Train on 189 samples, validate on 48 samples
Epoch 1/30
189/189 [==============================] - 12s 64ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 72.9696 - val_acc: 0.2083
Epoch 2/30
189/189 [==============================] - 10s 54ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 72.6355 - val_acc: 0.2083
Epoch 3/30
189/189 [==============================] - 10s 54ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 72.4304 - val_acc: 0.2083
Epoch 4/30
189/189 [==============================] - 10s 53ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 72.2593 - val_acc: 0.2083
Epoch 5/30
189/189 [==============================] - 10s 54ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 72.0964 - val_acc: 0.2083
Epoch 6/30
189/189 [==============================] - 11s 56ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 71.8247 - val_acc: 0.2083
Epoch 7/30
189/189 [==============================] - 11s 59ms/s